In [1]:
import pandas as pd
import numpy as np

# Methods

## Baseline

In [24]:
def apply_baseline_uncensoring(df):
    df_result = df.copy()
    df_result['Baseline_Demand'] = df_result['Verkauf']
    return df_result

## Naive Methods

In [25]:
def apply_n1_uncensoring(df):
    """
    N1: replace censored values with mean of all values within each POS.
    """
    df = df.copy()
    df['N1_Demand'] = df['Verkauf'].copy()

    # Identify closed observations
    is_closed = (df["Zensiert"] == 1)

    # Compute mean Verkauf per group and broadcast using transform
    group_means = df.groupby(['EHASTRA_EH_NUMMER'])['N1_Demand'].transform('mean').round()
    
    # Replace closed observations
    df.loc[is_closed, 'N1_Demand'] = group_means[is_closed]
    
    return df

def apply_n2_uncensoring(df):
    """
    N2: replace censored values with mean of uncensored values within each POS.
    """
    df = df.copy()
    df['N2_Demand'] = df['Verkauf'].copy()
    
    # # Calculate mean of uncensored observations for each group
    # uncensored_means = (
    #     df[df['Zensiert'] == 0]
    #     .groupby(['EHASTRA_EH_NUMMER'])['Verkauf']
    #     .mean()
    #     .rename('uncensored_mean')
    #     .reset_index()
    #     .round()
    # )
    
    # # Merge back to original DataFrame
    # df = df.merge(uncensored_means, on=['EHASTRA_EH_NUMMER'], how='left')
    
    # # CREATE MASK AFTER MERGE - this is the key fix
    # mask = (df['Zensiert'] == 1) & df['uncensored_mean'].notna()
    # df.loc[mask, 'N2_Demand'] = df.loc[mask, 'uncensored_mean']
    
    # return df.drop('uncensored_mean', axis=1)
    # mean only over uncensored values
    uncensored_means = (
        df.loc[df['Zensiert'] == 0]
        .groupby('EHASTRA_EH_NUMMER')['Verkauf']
        .transform('mean')
    )

    # align values directly back to df
    df.loc[df['Zensiert'] == 1, 'N2_Demand'] = (
        df.loc[df['Zensiert'] == 1, 'EHASTRA_EH_NUMMER']
        .map(df.loc[df['Zensiert'] == 0].groupby('EHASTRA_EH_NUMMER')['Verkauf'].mean().round())
    )
    return df


def apply_n3_uncensoring(df):
    """
    N3: replace censored values with max(current value, average of non-censored values) within each POS.
    """
    df = df.copy()
    df['N3_Demand'] = df['Verkauf'].copy()
    
    # Compute open group means and merge back
    open_means = (
        df[df['Zensiert'] == 0]
        .groupby('EHASTRA_EH_NUMMER')['Verkauf']
        .mean()
        .reset_index()
        .rename(columns={'Verkauf': 'open_mean'})
        .round()
    )
    
    df = df.merge(open_means, on='EHASTRA_EH_NUMMER', how='left')
    
    # CREATE MASK AFTER MERGE - this is the key fix
    mask = (df['Zensiert'] == 1) & df['open_mean'].notna()
    df.loc[mask, 'N3_Demand'] = np.maximum(
        df.loc[mask, 'Verkauf'],
        df.loc[mask, 'open_mean']
    )
    
    return df.drop('open_mean', axis=1)

## EM

In [26]:
from scipy.stats import poisson
def apply_em_uncensoring(df, max_iter=30, tolerance=1e-6):
    """
    Parameters:
    - max_iter: Maximum number of iterations
    - tolerance: Convergence tolerance
    """
    
    df = df.copy()
    df['EM_Demand'] = df['Verkauf'].copy()
    
    stockout_condition = (df['Zensiert'] == 1)
    
    for (pos,), group in df.groupby(['EHASTRA_EH_NUMMER']):
        try:
            group_stockout = stockout_condition.loc[group.index]
            
            if not group_stockout.any():
                continue
            
            sales = group['EM_Demand'].values
            is_stockout = group_stockout.values
            
            uncensored = sales[~is_stockout]
            censored = sales[is_stockout]
            
            # Skip if no uncensored data - but keep original values
            if len(uncensored) == 0:
                continue
            
            # Quick lambda initialization
            lambda_est = np.mean(uncensored) if len(uncensored) > 0 else np.mean(sales) * 1.5
            lambda_est = max(lambda_est, 0.1)
            
            # Fast EM loop
            for iteration in range(max_iter):
                lambda_old = lambda_est
                
                # Batch E-step
                surv_prob = 1 - poisson.cdf(censored - 1, lambda_est)
                exact_prob = poisson.pmf(censored, lambda_est)
                surv_prob = np.maximum(surv_prob, 1e-12)
                
                expected = lambda_est + censored * exact_prob / surv_prob
                expected = np.maximum(expected, censored.astype(float))
                
                # M-step
                lambda_est = max(np.mean(np.concatenate([uncensored, expected])), 0.1)
                
                if abs(lambda_est - lambda_old) < tolerance:
                    break
            
            # Final update
            surv_prob = 1 - poisson.cdf(censored - 1, lambda_est)
            exact_prob = poisson.pmf(censored, lambda_est)
            surv_prob = np.maximum(surv_prob, 1e-12)
            
            final_expected = lambda_est + censored * exact_prob / surv_prob
            final_expected = np.maximum(final_expected, censored.astype(float))
            
            # Update original dataframe
            stockout_indices = group.index[is_stockout]
            df.loc[stockout_indices, 'EM_Demand'] = final_expected.round()
            
        except Exception as e:
            print(f"EM error for POS {pos}: {e}")
            # Fill with original Verkauf values for this POS when error occurs
            group_stockout_indices = group.index[stockout_condition.loc[group.index]]
            df.loc[group_stockout_indices, 'EM_Demand'] = df.loc[group_stockout_indices, 'Verkauf']
            continue
    
    # Replace any NaN values with original Verkauf
    nan_mask = df['EM_Demand'].isna()
    df.loc[nan_mask, 'EM_Demand'] = df.loc[nan_mask, 'Verkauf']
    
    return df

## PD

In [27]:
from scipy.stats import poisson
def apply_pd_uncensoring(df, tau=0.5, max_iter=20, tolerance=1e-4):
    """PD with skip for invalid POS groups and fallback to original Verkauf"""
    df = df.copy()
    df["is_closed"] = (df["Zensiert"] == 1)
    df['PD_Demand'] = df['Verkauf'].copy().astype(float)

    def compute_pd_projection1(obs_val, lambda_est, tau):
        """Your PD projection with NaN protection"""
        try:
            # Check for NaN inputs
            if pd.isna(obs_val) or pd.isna(lambda_est) or lambda_est <= 0:
                return float(obs_val) if not pd.isna(obs_val) else 0.0
            
            obs_val = int(round(obs_val))
            
            def objective(k_proj):
                k_proj = int(round(k_proj))
                
                if k_proj < obs_val:
                    return float('inf')
                
                # Area A: P(obs_val <= X <= k_proj)
                area_A = poisson.cdf(k_proj, lambda_est) - poisson.cdf(obs_val - 1, lambda_est)
                
                # Area B: P(X > k_proj)
                area_B = 1 - poisson.cdf(k_proj, lambda_est)
                
                if area_B > 1e-10:
                    ratio = area_A / area_B
                    target_ratio = (1 - tau) / tau
                    return abs(ratio - target_ratio)
                else:
                    return abs(area_A - (1 - tau))
            
            # Search for optimal projection
            upper_bound = int(obs_val + max(10, int(3 * np.sqrt(lambda_est))))
            
            best_k = obs_val
            best_objective = float('inf')
            
            for k in range(int(obs_val), upper_bound + 1):
                obj_val = objective(k)
                if obj_val < best_objective:
                    best_objective = obj_val
                    best_k = k
            
            return float(best_k)
            
        except Exception:
            # Fallback to original value
            return float(obs_val) if not pd.isna(obs_val) else 0.0
    
    # Process groups
    grouped = df.groupby('EHASTRA_EH_NUMMER')
    
    for pos, group in grouped:
        try:
            open_mask = ~group['is_closed']
            closed_mask = group['is_closed']
            
            open_sales = group.loc[open_mask, 'PD_Demand'].values
            closed_sales = group.loc[closed_mask, 'PD_Demand'].values
            
            # SKIP POS WITHOUT VALID UNCENSORED DATA
            if len(open_sales) == 0 or np.all(pd.isna(open_sales)):
                # print(f"Skipping POS {pos}: no valid uncensored data")
                continue
            
            # Initialize lambda parameter
            lambda_est = np.mean(open_sales[~pd.isna(open_sales)])
            
            # SKIP IF LAMBDA IS INVALID
            if pd.isna(lambda_est) or lambda_est <= 0:
                print(f"Skipping POS {pos}: invalid lambda {lambda_est}")
                continue
            
            lambda_est = max(lambda_est, 0.1)
            closed_indices = group[closed_mask].index.values
            
            # Iterative process
            for iteration in range(max_iter):
                lambda_old = lambda_est
                
                # Project closed observations
                projected_values = np.array([
                    compute_pd_projection1(obs, lambda_est, tau)
                    for obs in closed_sales
                ])
                
                # Re-estimate lambda
                all_values = np.concatenate([open_sales, projected_values])
                lambda_est = np.mean(all_values)
                lambda_est = max(lambda_est, 0.1)
                
                # Check convergence
                if abs(lambda_est - lambda_old) < tolerance:
                    break
            
            # Final projection
            final_projections = np.array([
                compute_pd_projection1(obs, lambda_est, tau)
                for obs in closed_sales
            ])
            
            # Update dataframe
            df.loc[closed_indices, 'PD_Demand'] = final_projections.round()
            
        except Exception as e:
            print(f"PD error for POS {pos}: {e}")
            # FALLBACK: Keep original values for this POS
            continue
    
    # FINAL FALLBACK: Any remaining NaN values get original Verkauf
    nan_mask = df['PD_Demand'].isna()
    df.loc[nan_mask, 'PD_Demand'] = df.loc[nan_mask, 'Verkauf']
    
    return df.drop('is_closed', axis=1)

## Conrad

In [37]:
from scipy.stats import poisson

def berechnung(links, rechts, n, N, r, x_summe, value_tol=0.00001, max_iterations=1000):
    """
    Till's Code
    """
    iteration = 0
    
    while iteration < max_iterations:
        mu = (links + rechts) / 2
        wert_0 = (x_summe - mu * n) * (1 - poisson.cdf(N-1, mu)) + mu * (n - r) * (1 - poisson.cdf(N-2, mu))
        
        # if iteration < 3:
            #print(f"Iter {iteration}: mu={mu:.4f}, wert_0={wert_0:.8f}")
        
        if abs(wert_0) < value_tol:
            #print(f"Converged after {iteration} iterations: mu={mu:.4f}, wert_0={wert_0:.8f}")
            return mu
        elif wert_0 < 0:
            rechts = mu
        elif wert_0 > 0:
            links = mu
            
        iteration += 1
    
    #print(f"✗ Max iterations reached: mu={mu:.4f}")
    return mu

def test_conrad_example():
    links = 1
    rechts = 100
    n = 13
    N = 10
    r = 7
    x_summe = 58
    
    print(f"links={links}, rechts={rechts}")
    print(f"n={n}, N={N}, r={r}, x_summe={x_summe}")
    
    result = berechnung(links, rechts, n, N, r, x_summe)
    print(f"Result: μ = {result:.4f}")
    print(f"Expected: μ ≈ 10.18")
    
    return result

def create_order_specific_mu_dict(df):

    order_specific_mu_dict = {}
    
    for (year, week), week_data in df.groupby(['Heftjahr', 'Heftnummer']):
        for bezug_val, group in week_data.groupby('Bezug'):
            n = len(group)
            N = bezug_val
            
            # Count non-stockouts
            stockouts_mask = (group['Zensiert'] == 1)
            r = n - stockouts_mask.sum()  # r = number of NON-stockouts
            
            # x_summe = sum of UNCENSORED observations only
            uncensored_sales = group[~stockouts_mask]['Verkauf']
            x_summe = uncensored_sales.sum()
            
            # Skip problematic cases
            if n < 3:
                continue
            if r == n:  # No stockouts = no censoring information
                continue
            if r == 0:  # All stockouts = no uncensored observations
                continue
            
            try:
                links = 1
                rechts = 100
                mu_est = berechnung(links, rechts, n, N, r, x_summe)
                if mu_est:
                    key = (year, week, bezug_val)
                    order_specific_mu_dict[key] = mu_est
            except Exception as e:
                print(f"Error in week {week}, Bezug {N}: {e}")
                continue

    #print(f"Successfully estimated μ for {len(order_specific_mu_dict)} groups")
    return order_specific_mu_dict

def expected_poisson_tail(mu, N, max_k=200):
    """
    Compute E[X | X >= N] for X ~ Poisson(mu)
    """
    k_vals = np.arange(N, max_k)
    pmf = poisson.pmf(k_vals, mu)
    tail_prob = 1 - poisson.cdf(N - 1, mu)
    if tail_prob < 1e-8:
        return N  # fallback: don't uncensor
    return np.sum(k_vals * pmf) / tail_prob

# Internal function
def apply_conrad_uncensoring_1(df, order_specific_mu_dict):
    """
    Given a DataFrame with Verkauf, Bezug, is_stockout, Heftjahr, Heftnummer,
    replace Verkauf with E[X | X >= Bezug] when censored.
    """
    df = df.copy()
    df['Conrad_Demand'] = df['Verkauf'].copy()

    for idx, row in df.iterrows():
        # Skip if no stockout occurred
        if row['Zensiert'] == 0:
            continue
            
        # Get the order-quantity-specific demand parameter
        key = (row['Heftjahr'], row['Heftnummer'], row['Bezug'])
        mu = order_specific_mu_dict.get(key, None)

        if mu is None:
            # no estimate available for this specific (week, order_quantity) — keep original value
            continue

        # POS sold out — uncensor using the specific distribution for this order quantity
        est_demand = expected_poisson_tail(mu, row['Bezug'])
        df.at[idx, 'Conrad_Demand'] = np.round(est_demand)

    return df

# Wrapper function for script
def apply_conrad_uncensoring(df):
    """
    WRAPPER FUNCTION: This is what gets called by the main processing loop
    """
    # Step 1: Create mu dictionary from the dataset
    order_specific_mu_dict = create_order_specific_mu_dict(df)
    
    # Step 2: Apply uncensoring using the mu dictionary
    return apply_conrad_uncensoring_1(df, order_specific_mu_dict)

## Nahmias

In [29]:
from scipy.stats import norm

def nahmias_estimation(sales, S):
    """
    Nahmias method for censored normal data
    """
    sales = np.array(sales)
    n = len(sales)
    
    observed = sales[sales < S]
    r = len(observed)
    p = r / n
    
    if r < 2 or r >= n-1 or p <= 0 or p >= 1:
        return None, None
    
    try:
        x_bar = np.mean(observed)
        s2 = np.var(observed, ddof=1)
        z = norm.ppf(p)
        
        sigma_hat2 = s2 / (1 - (z * norm.pdf(z) / p) - (norm.pdf(z)**2 / p**2))
        sigma_hat = np.sqrt(sigma_hat2)
        mu_hat = x_bar + sigma_hat * norm.pdf(z) / p
        
        if not np.isfinite(mu_hat) or not np.isfinite(sigma_hat) or sigma_hat <= 0:
            return None, None
            
        return mu_hat, sigma_hat
        
    except Exception:
        return None, None

def create_order_specific_nahmias_dict(df):
    """
    Create μ and σ estimates for each (week, order_quantity) combination
    """
    order_specific_mu_dict = {}
    order_specific_sigma_dict = {}

    for (year, week), week_data in df.groupby(['Heftjahr', 'Heftnummer']):
        for bezug_val, group in week_data.groupby('Bezug'):
            n = len(group)
            S = bezug_val
            
            if n < 5:
                continue
            
            sales = group['Verkauf'].values
            
            try:
                mu_est, sigma_est = nahmias_estimation(sales, S)
                if mu_est is not None and sigma_est is not None:
                    key = (year, week, bezug_val)
                    order_specific_mu_dict[key] = mu_est
                    order_specific_sigma_dict[key] = sigma_est
            except Exception as e:
                continue

    # print(f"Successfully estimated μ,σ for {len(order_specific_mu_dict)} groups")
    return order_specific_mu_dict, order_specific_sigma_dict

def expected_normal_tail(mu, sigma, S):
    """
    Compute E[X | X >= S] for X ~ Normal(mu, sigma)
    """
    if sigma <= 0:
        return S
    
    z = (S - mu) / sigma
    
    if z > 6:
        return S
    
    tail_prob = 1 - norm.cdf(z)
    
    if tail_prob < 1e-10:
        return S
    
    expected_value = mu + sigma * norm.pdf(z) / tail_prob
    
    return expected_value

def apply_nahmias_uncensoring_1(df, order_specific_mu_dict, order_specific_sigma_dict):
    """
    Uncensor dataset using Nahmias estimates
    """
    #df = df.copy()
    df['Nahmias_Demand'] = df['Verkauf'].copy()

    for idx, row in df.iterrows():
        if row['Zensiert'] == 0:
            continue
            
        key = (row['Heftjahr'], row['Heftnummer'], row['Bezug'])
        mu = order_specific_mu_dict.get(key, None)
        sigma = order_specific_sigma_dict.get(key, None)

        if mu is None or sigma is None:
            continue

        est_demand = expected_normal_tail(mu, sigma, row['Bezug'])
        df.at[idx, 'Nahmias_Demand'] = np.round(est_demand)

    return df

def test_nahmias():
    """Test implementation"""
    mu_true = 100
    sigma_true = 30
    S = 110
    n = 100
    
    np.random.seed(42)
    demand = np.random.normal(mu_true, sigma_true, n)
    sales = np.minimum(demand, S)
    
    print("Testing Nahmias implementation:")
    print(f"True μ: {mu_true}, True σ: {sigma_true}")
    print(f"S (censoring limit): {S}")
    print(f"Sample size: {n}")
    
    mu_hat, sigma_hat = nahmias_estimation(sales, S)
    
    naive_mean = np.mean(sales)
    naive_std = np.std(sales, ddof=1)
    
    print(f"True mean: {mu_true}")
    print(f"Naive mean (sales): {naive_mean:.2f}")
    print(f"Corrected estimator (Nahmias): {mu_hat:.2f}")
    print(f"True Std.Dev.: {sigma_true}")
    print(f"Corrected Std.Dev.: {sigma_hat:.2f}")
    print(f"Naive Std.Dev. (sales): {naive_std:.2f}")
    
    return mu_hat, sigma_hat

def apply_nahmias_uncensoring(df):
    """
    WRAPPER FUNCTION: This is what gets called by the main processing loop
    """
    # Step 1: Create mu dictionary from the dataset
    order_specific_mu_dict, compute_mu_sigma_nahmias = create_order_specific_nahmias_dict(df)
    
    # Step 2: Apply uncensoring using the mu dictionary
    return apply_nahmias_uncensoring_1(df, order_specific_mu_dict, compute_mu_sigma_nahmias)

## Agrawal

In [30]:
from scipy.stats import nbinom
from scipy.optimize import minimize_scalar
from concurrent.futures import ThreadPoolExecutor, as_completed

def find_p(N, bezug, F_observed):
    """"
    Find p for a given N such that P(X < S) from the Negative Binomial
    matches the empirically observed uncensored probability.
    """
    func = lambda p: abs(nbinom.cdf(bezug - 1, N, p) - F_observed)
    res = minimize_scalar(func, bounds=(0.01, 0.99), method='bounded')
    return res.x


def uncensor(N_hat, p_hat, s, max):
    '''
    Compute E[X|X>=s] for X~NB(N_hat,p_hat).
    max: upper demand limit used in calculation
    '''
    X_vals = np.arange(s, max)
    pmf = nbinom.pmf(X_vals, N_hat, p_hat)
    tail_prob = 1- nbinom.cdf(s-1, N_hat, p_hat)
    if tail_prob < 1e-8:
        return s  # fallback: don't uncensor
    exp = np.sum(X_vals * pmf) / tail_prob
    return int(np.round(exp))


def compute_NB_result(group_key_data):
    '''
    Given one hierarchical group, calculate the Agrawal demand
    '''
    def objective(N, bezug, x_bar, F_observed):
        p = find_p(N, bezug, F_observed)
        prob_uncensored = nbinom.cdf(bezug - 1, N, p)
        if prob_uncensored == 0.0: return 10**8 #if probability of demand being uncensored is too low, return a high objective value
        expect = nbinom.expect(lambda x: x, args=(N, p), lb=0, ub=bezug-1)
        mean_uncensored = expect / prob_uncensored
        return abs(mean_uncensored - x_bar)
    

    (bezug, period), group = group_key_data
    group = group.dropna(subset=['Bezug', 'Verkauf'])

    group_uncensored = group[group['Zensiert']==0]
    x_bar = group_uncensored['Verkauf'].mean()
    F_observed = len(group_uncensored)/len(group)

    result = minimize_scalar(lambda N: objective(N,bezug,x_bar,F_observed), bounds=(0.1, 500), method='bounded')
    N_hat = result.x
    p_hat = find_p(N_hat, bezug, F_observed)

    ub = max(100, bezug*2)
    demand = uncensor(N_hat, p_hat, bezug, max = ub)
    #print(f'Period: {period}, Bezug: {bezug}, N: {N_hat}, p: {p_hat}')
    return {'Period': period, 'Bezug': bezug, 'Agrawal_Demand': demand}


def apply_agrawal_uncensoring(df, max_cpus=1):
    '''
    Group magazine data by bezug and period run Agrawal algorithm in parallel.
    Returns original dataframe with new columns Agrawal_Demand
    '''
    #Step 1: calculate Agrawal demand
    df.sort_values(['Bezug', 'Period'], inplace=True)
    grouped = list(df.groupby(['Bezug', 'Period']))
    results = []

    # with ThreadPoolExecutor(max_workers=max_cpus) as executor:
    #     futures = [executor.submit(compute_NB_result, group) for group in grouped]
    #     for future in as_completed(futures):
    #         results.append(future.result())
    for group in grouped:
        results.append(compute_NB_result(group))
    df_results= pd.DataFrame(results)

    #Step 2: calculate hierarchical group uncensored fraction)
    group_data = []
    for (bezug,period),group in df.groupby(['Bezug', 'Period']):
        group_data.append({'Bezug':bezug, 'Period':period, 'Uncensored_fraction': len(group[group['Zensiert']==0])/len(group)})
    df_group = pd.DataFrame(group_data)
    df_results = pd.merge(df_results, df_group, how='outer', on=['Bezug','Period'])

    #Step 3: add results to original data
    df_join = pd.merge(df, df_results, how="outer", on=['Bezug','Period'])

    #only use Agrawal estimation for censored data (use real sales value for not censored data)
    df_join['Agrawal_Demand'] = np.where(df_join['Zensiert'], df_join['Agrawal_Demand'], df_join['Verkauf'])

    #only use Agrawal estimation for valid results (uncensored fraction is not 0 or 1)
    df_join['Agrawal_Demand'] = np.where((df_join['Uncensored_fraction']==0.0)|(df_join['Uncensored_fraction']==1.0), df_join['Verkauf'], df_join['Agrawal_Demand'])
    
    df_join.drop(['Uncensored_fraction'],axis='columns', inplace=True)
    return df_join

## Bayesian

In [ ]:
from scipy.stats import nbinom

### Precomputations

To improve runtime of Bayesian method, precompute PMF and CMF of negative binomial distribution. Also precompute conditional sales probabilities.<br>
note: $p_{idx}$ corresponds to the index in the grid $0.900, 0.901, ..., 0.990$
<br><br>
pmf_table[$n,r,p_{idx}$] = nbinom.pmf($n,r,p$) where $p = 0.900+p_{idx}*0.001$<br>
cmf_table[$n,r,p_idx$] = nbinom.cmf($n,r,p$) where $p = 0.900+p_{idx}*0.001$<br>
cond_pmf_sales[$X, s, r, p_{idx}$] = P($X$ sales with $s$ inventory and demand ~ NB($r,p$)) where $p = 0.900+p_{idx}*0.001$<br>
<br>
We choose to precompute above values up to $n=200, r=300, X=200, s=200$ as these upper bounds will cover most data points. For remaining cases not included in the tables, we directly compute them.
<br>
If you do not want to use precomputations, simply set the max_n and max_r variables to $-1$.

In [7]:
#upper bounds for initial NB precomputing
max_n = 200
max_r = 300

#non-parametric p distribution values for NB fitting
p_grid = np.linspace(0.900, 0.990, 91)

def precompute_NB ():
    '''
    Calculates tables containing NB pmf and cmf values up to given bounds.
    Output: two tables with shape (max_n, max_r, len(p_grid))
    '''
    pmf_table = np.zeros((max_n+1, max_r +1, len(p_grid)))
    cmf_table = np.zeros((max_n+1, max_r +1, len(p_grid)))
    for p_idx, p in enumerate(p_grid):
        for r in range(max_r+1):
            pmf_table[:, r, p_idx] = nbinom.pmf(np.arange(0,max_n+1), r, p)
            cmf_table[:, r, p_idx] = nbinom.cdf(np.arange(0,max_n+1), r, p)
    np.save("pmf_table.npy", pmf_table)
    np.save("cmf_table.npy", cmf_table)

def precompute_cond_pmf_sale():
    '''
    Calculates table containing conditional pmf sales (given demand~NB(r,p) and inventory is s)
    For uncensored sales (X < s): table returns nbinom.pmf(X, r, p)
    For censored sales (X >= s): table returns 1 - nbinom.cmf(s-1, r, p)

    Output: one table with shape (max_n, max_n, max_r, len(p_grid))
    Specifically, value at index (X, s, r, p) represents probability of getting X sales given demand~NB(r,p) and inventory is s
    '''
    cond_pmf_sale = np.zeros((max_n+1, max_n+1, max_r+1, len(p_grid)))

    for p_idx, p in enumerate(p_grid):
        for X in range(max_n+1):
            for s in range(max_n+1):
                for r in range(max_r+1):
                    cond_pmf_sale[X, s, r, p_idx] = pmf_table[X, r, p_idx] if X < s else 1 - cmf_table[s-1, r, p_idx]
    np.save("cond_pmf_sale_table.npy", cond_pmf_sale)

#precompute_NB()
pmf_table = np.load('pmf_table.npy')
cmf_table = np.load('cmf_table.npy')
#precompute_cond_pmf_sale()
cond_pmf_sales_table = np.load('cond_pmf_sale_table.npy')

print(pmf_table.shape) # (201,301,91)
print(cmf_table.shape) # (201,301,91)
print(cond_pmf_sales_table.shape) # (201,201,301,91)

(201, 301, 91)
(201, 301, 91)
(201, 201, 301, 91)


### Bayesian Algorithm

In [ ]:
def quantile_quantity(r, c_ratio, pmf_p, lo=0, hi=100):
    '''
    Returns minimum order quantity such that predicted demand is above the critical ratio
    '''
    # Use binary search to determine lowest value such that quantile > critical ratio
    # quantile at mid = ∑_(demand = 0 to mid)∑_(p_grid)  pmf(demand, r, p)*π(p)
    while lo < hi:
        mid = (lo + hi) // 2
        
        if mid > max_n or r > max_r:
            quantile =  np.sum([np.dot(nbinom.pmf(y, r, p_grid), pmf_p) for y in range(int(mid) + 1)])
        else:
            quantile = np.sum([np.dot(pmf_table[y, r, :], pmf_p) for y in range(int(mid) + 1)])

        if quantile < c_ratio:
            lo = mid + 1
        else:
            hi = mid
    return lo

def uncensor(r, pmf_p, s, max):
    '''
    Calculate E[X|X>=s, X~NB(r, p)] with p distribution = pmf_p
    max: upper demand limit used in calculation
    '''
    # tail_prob = probability of demand >=s = ∑_(p_grid) sf(s, r, p)*π(p)
    # pmf = pmf of demand s, s+1, ..., max (entry for demand x = ∑_(p_grid) pmf(x, r, p)*π(p))
    if r > max_r or max > max_n:
        pmf = np.dot(nbinom.pmf(np.arange(s,max+1)[:, None], r, p_grid), pmf_p) #length = max+1-s
        tail_prob = np.dot(1 - nbinom.cdf(s-1, r, p_grid)[None, :], pmf_p).item()
    else:
        pmf = np.dot(pmf_table[s:max+1, r,:], pmf_p)
        tail_prob = np.dot(1 - cmf_table[s-1, r,:], pmf_p)

    if tail_prob < 1e-8:
        return s  # fallback: don't uncensor (demand>=s is unlikely)
    
    exp = np.dot(np.arange(s,max+1), pmf)/tail_prob
    return int(np.round(exp))

def compute_bayesian_result(group_key_data, c_ratio):
    '''
    given one pos group and critical ratio, calculate the bayesian demand and order quantity
    if there is no 2022-2023 data or if mean 2022-2023 sales is too low (<0.052), this pos is skipped
    '''
    (pos,), group = group_key_data
    group = group.dropna(subset=['Bezug', 'Verkauf'])
    results = []

    #determine appropriate r value using 2022-2023 mean sales
    train_data = group[group['Heftjahr']<2024]
    if len(train_data) == 0:
        #print(f'skipped, {pos} has no 2022-2023 data points')
        return
    else:
        mean_2223 = train_data['Verkauf'].mean()
        r = int(mean_2223*0.95/0.05) #NB(r, 0.95) has mean ≈ mean(2022-2023 sales)
        if r == 0.0:
            print(f'Bayesian skipped for {pos}, r = 0')
            return #don't use Bayesian for this POS

    # initialize distribution of p, π, to Uniform
    p_grid = np.linspace(0.900, 0.990, 91)
    pmf_p = np.full(len(p_grid), 1.0 / len(p_grid))

    #update π with each new verkauf-bezug data point
    for row in group.itertuples():
        #cond_sale_probs = probability of sale given demand~NB(r,p) and inventory level
        if row.Bezug > max_n or r > max_r:
            cond_sale_probs = [nbinom.pmf(row.Verkauf, r, p) if row.Verkauf < row.Bezug else 1-nbinom.cdf(row.Bezug - 1, r, p) for p in p_grid] #shape = len(p_grid)
        else:
            cond_sale_probs = cond_pmf_sales_table[int(row.Verkauf), int(row.Bezug), r, :]  #shape = len(p_grid)
            
        sale_prob = np.dot(cond_sale_probs, pmf_p) #summed over all p values
        if pmf_p.size == 0:
            print(pmf_p)
            print(row)
        #do not update π if probability of sale is too low
        if sale_prob != 0.0:
            pmf_p = pmf_p * cond_sale_probs / sale_prob
        
        ub = max(50,2*int(train_data['Verkauf'].max()))
        Q = quantile_quantity(r, c_ratio, pmf_p, lo=0, hi=ub)
        if Q == ub: print(f'warning: {pos}, {row.Period}, Q is limited by upper bound of {Q}') #check if upper bound is limiting Q output

        if row.Zensiert == 1: demand = uncensor(r, pmf_p, int(row.Bezug), max= ub)
        else: demand = row.Verkauf

        results.append({'EHASTRA_EH_NUMMER': pos, 'Period':row.Period, 'Bayesian_Demand': demand, 'Bayesian_Q': Q})

    #print(f'{pos}, r = {r}, final pmf_p: {pmf_p}')
    return pd.DataFrame(results)


def apply_bayesian_uncensoring_optimization(df, c_ratio=0.9, max_cpus = 1):
    '''
    Group magazine data by pos and run bayesian algorithm in parallel.
    Returns original dataframe with new columns Bayesian_Demand, and Bayesian_Q
    '''
    #Step 1: calculate Bayesian Demand and Q values
    df.sort_values(['EHASTRA_EH_NUMMER', 'Period'], inplace=True)
    grouped = df.groupby(['EHASTRA_EH_NUMMER'])
    grouped = list(grouped)
    results = []

    for group in grouped:
        bayesian_result = compute_bayesian_result(group, c_ratio=0.9)
        if bayesian_result is not None: results.append(bayesian_result)
    df_Bayesian = pd.concat(results)

    #Step 2: add results to original dataframe
    df_join = pd.merge(df, df_Bayesian, how="outer", on=['EHASTRA_EH_NUMMER', 'Period'])
    df_join.set_index(['EHASTRA_EH_NUMMER', 'Period'])

    # Step 3: For POS where Bayesian does not work, fill in missing demand predictions with verkauf. Missing Q values are not changed
    df_join.loc[df_join['Verkauf'].notna() & df_join['Bayesian_Demand'].isna(), 'Bayesian_Demand'] = df_join['Verkauf']
    return df_join

# Run Methods

In [45]:
def apply_uncensoring(methods, dataframes):
    '''
    methods: list of uncensoring methods to use
    dataframes: list of original data (magazines) to use
    Applies each method to all dataframes and saves a csv file for each method (original columns + method_Demand)
    '''
    method_functions = {
        'N1': apply_n1_uncensoring,
        'N2': apply_n2_uncensoring,
        'N3': apply_n3_uncensoring,
        'EM': apply_em_uncensoring,
        'PD': apply_pd_uncensoring,
        'Nahmias': apply_nahmias_uncensoring,
        'Conrad': apply_conrad_uncensoring,
        'Baseline': apply_baseline_uncensoring,
        'Bayesian': apply_bayesian_uncensoring_optimization,
        'Agrawal': apply_agrawal_uncensoring
    }
    df_all_uncensoring = pd.DataFrame()
    for dataframe in dataframes:
        df_magazine = None
        for method in methods:
            print(f'Running uncensoring with method {method} on magazine {dataframe['Magazine'].unique()[0]}')
            df = method_functions[method](dataframe)
            if df_magazine is None:
                df_magazine = df
            else:
                new_cols = [c for c in df.columns if c not in df_magazine.columns or c in ['Period','EHASTRA_EH_NUMMER']]
                df_magazine = pd.merge(df_magazine, df[new_cols], how = 'outer', on = ['Period', 'EHASTRA_EH_NUMMER'])
        df_all_uncensoring = pd.concat([df_all_uncensoring, df_magazine], ignore_index=True)
    
    return df_all_uncensoring

In [46]:
#demo, only use first few pos per magazine
#skip 'Agrawal' for now, runs slowly and poor results
uncensoring_methods = ['N1', 'N2', 'N3', 'EM', 'PD', 'Nahmias', 'Conrad','Bayesian', 'Baseline']
original_data = []

for letter in 'ABCDEFGHI':
    df_magazine = pd.read_csv('original_data/'+letter+'_20250212_ZQ0.35_ZG0.4_testfile.csv')# adjust this accordingly
    df_magazine['Magazine'] = letter
    df_magazine = df_magazine[df_magazine["EHASTRA_EH_NUMMER"].str.extract(r"(\d+)", expand=False).astype(int) < 24000]
    print(letter, df_magazine['EHASTRA_EH_NUMMER'].unique())
    print(len(df_magazine))
    original_data.append(df_magazine)

df_results = apply_uncensoring(uncensoring_methods, original_data)
df_results.to_csv('Uncensoring_results.csv', index=False)# adjust this accordingly

A ['EHA0017186' 'EHC0004006' 'EHC0005070' 'EHH0010681' 'EHB0023505'
 'EHB0019844' 'EHE0003391' 'EHA0021360']
1056
B ['EHA0020265' 'EHD0015991' 'EHB0019622' 'EHB0023505' 'EHG0003391']
660
C ['EHB0023505']
132
D ['EHA0021360' 'EHB0019844' 'EHH0010681']
177
E ['EHB0019622' 'EHC0005070' 'EHE0003391']
99
F ['EHG0006538']
62
G ['EHB0023505' 'EHG0006538']
132
H ['EHB0019622' 'EHB0023505' 'EHG0006538' 'EHH0011851' 'EHI0022887']
140
I ['EHA0020265' 'EHH0010681' 'EHH0011851']
54
Running uncensoring with method N1 on magazine A
Running uncensoring with method N2 on magazine A
Running uncensoring with method N3 on magazine A
Running uncensoring with method EM on magazine A
Running uncensoring with method PD on magazine A
Running uncensoring with method Nahmias on magazine A
Running uncensoring with method Conrad on magazine A
Running uncensoring with method Bayesian on magazine A
Running uncensoring with method Baseline on magazine A
Running uncensoring with method N1 on magazine B
Running uncenso

In [ ]:
uncensoring_methods = ['N1', 'N2', 'N3', 'EM', 'PD', 'Nahmias', 'Conrad', 'Baseline', 'Bayesian', 'Agrawal']
original_data = []

for letter in 'ABCDEFGHI':
    df_magazine = pd.read_csv('original_data/'+letter+'_20250212_ZQ0.35_ZG0.4_testfile.csv')# adjust this accordingly
    df_magazine['Magazine'] = letter
    original_data.append(df_magazine)

df_results = apply_uncensoring(uncensoring_methods, original_data)
df_results.to_csv('Uncensoring_results.csv', index=False)# adjust this accordingly